In [1]:

%pprint
import sys
sys.path.insert(1, '../py')

Pretty printing has been turned OFF


In [2]:

from FRVRS import nu, fu
from pandas import read_excel, to_datetime

In [3]:

# load data frames
data_frames_list = nu.load_data_frames(frvrs_logs_df='frvrs_logs_df')
frvrs_logs_df = data_frames_list['frvrs_logs_df']
print(frvrs_logs_df.shape) # (829116, 125)

Attempting to load /mnt/c/Users/DaveBabbitt/Documents/GitHub/itm-analysis-reporting/saves/pkl/frvrs_logs_df.pkl.
(829116, 125)


In [4]:

if nu.pickle_exists('learner_types_df'): learner_types_df = nu.load_object('learner_types_df')
else:
    file_path = '../data/xlsx/uuid_Master_with_Site_and_Learner_Type20240213.xlsx'
    learner_types_df = read_excel(file_path)
    learner_types_df.columns = ['session_file_date', 'session_file_name', 'session_uuid', 'responder_name', 'responder_type', 'site_name', 'encounter_layout']
    learner_types_df = learner_types_df.dropna(axis='columns', how='all')
    if ('session_file_date' in learner_types_df.columns): learner_types_df['session_file_date'] = to_datetime(learner_types_df['session_file_date'], infer_datetime_format=True)
    print(learner_types_df.shape) # (346, 6)
    nu.store_objects(learner_types_df=learner_types_df)
    nu.save_data_frames(learner_types_df=learner_types_df)

(346, 6)
Pickling to /mnt/c/Users/DaveBabbitt/Documents/GitHub/itm-analysis-reporting/saves/pkl/learner_types_df.pkl
Saving to /mnt/c/Users/DaveBabbitt/Documents/GitHub/itm-analysis-reporting/saves/csv/learner_types_df.csv


In [5]:

learner_types_df

,session_file_date,session_uuid,responder_name,responder_type,site_name,encounter_layout
0,2022-12-07 08:13:00,e77c15e0-d0de-413d-a471-20a285e81387,Will K,EMT-Basic,DCEMS-RND 1,DCEMS 11
1,2022-12-07 08:40:00,a9804531-1ed7-4b97-b3c1-96784faf5228,Kali S,Paramedic,DCEMS-RND 1,DCEMS 11
2,2022-12-07 10:11:00,7fcfbc5e-d5d2-431a-98ab-c3d9375e934e,Tim A,Paramedic,DCEMS-RND 1,DCEMS 11
3,2022-12-07 10:31:00,5ebf3067-e1a6-4711-810d-60095452c24c,J Moore,Paramedic,DCEMS-RND 1,DCEMS 11
4,2022-12-07 12:31:00,6f1a94ad-99cb-49a6-86f8-145e6c090fae,Brad F,Paramedic,DCEMS-RND 1,DCEMS 11
...,...,...,...,...,...,...
341,2023-03-15 13:03:42,4bac0722-bc76-4418-88f0-ccbbba469c5a,Robert R,EM-RES1,OSUCOM,First 11
342,2023-03-15 13:27:41,61dd1584-76e8-4536-9aeb-15276e646ff4,Elizabeth H,EM-RES2,OSUCOM,First 11
343,2023-03-15 13:49:31,dd601aee-8360-41f2-b89d-c37f3eb2c617,Abby Z,M4-FM,OSUCOM,First 11
344,2023-03-15 13:52:47,66cc58aa-8f53-4d06-b129-879fb3b08239,Trent P,EM-RES1,OSUCOM,First 11



# Create a DataFrame of all the Scenario Stats

In [6]:

# Get the columns that consistently have only one value in them per session
single_value_cols_set = set(frvrs_logs_df.columns)
for session_uuid, session_df in frvrs_logs_df.groupby('session_uuid'):
    
    # Find columns with a single unique value
    single_value_cols = set([col for col in session_df.columns if session_df[col].nunique() == 1])
    single_value_cols_set = single_value_cols_set.intersection(single_value_cols)
single_value_cols_set

{'is_a_one_triage_file', 'file_name', 'is_scene_aborted', 'logger_version', 'session_uuid'}

In [7]:

if nu.pickle_exists('file_stats_df'): file_stats_df = nu.load_object('file_stats_df')
else:
    file_stats_df = frvrs_logs_df[single_value_cols_set].merge(learner_types_df, how='left', on='session_uuid').drop_duplicates().reset_index(drop=True)
    nu.store_objects(file_stats_df=file_stats_df)
    nu.save_data_frames(file_stats_df=file_stats_df)
print(file_stats_df.shape) # (429, 10)

(429, 10)


In [8]:

file_stats_df.sample(8).T

,278,234,57,158,169,208,47,284
file_name,Double runs removed/22.12.01.1546r.csv,Disaster Day 2022/JD_0147.csv,All CSV files renamed by date/04.19.23.1044.csv,DCEMS Round 2 only triage sessions/2703ca93-17...,DCEMS Round 2 only triage sessions/638950ca-b0...,DCEMS Round 2 only triage sessions/Galen S..csv,All CSV files renamed by date/03.15.23.1303.csv,v.1.0/Clean 158e6365-673b-4030-8b36-6704be5996...
is_a_one_triage_file,True,True,True,True,False,True,True,True
logger_version,1.0,1.3,1.3,1.3,1.0,1.0,1.3,1.0
is_scene_aborted,False,False,False,False,False,False,False,False
session_uuid,91a84c6a-ba27-4116-8644-e5373b3588de,9cd41444-4813-45a8-81d0-a1d068184c8e,f3228193-b6dc-41fa-a2ab-1c5085046347,2703ca93-175b-4351-b0be-c67e23438d8f,638950ca-b008-4b14-bd07-933e16e7d8e0,77c15654-dbf5-46da-8e84-fe418cdf1c59,4bac0722-bc76-4418-88f0-ccbbba469c5a,158e6365-673b-4030-8b36-6704be5996a2
session_file_date,NaT,NaT,2023-04-19 10:43:00,NaT,2023-08-02 09:56:00,2023-07-26 10:32:00,2023-03-15 13:03:42,NaT
responder_name,NaN,NaN,Andy V,NaN,Ryan S,Galen S,Robert R,NaN
responder_type,NaN,NaN,Paramedic,NaN,Paramedic,Paramedic,EM-RES1,NaN
site_name,NaN,NaN,Madison Twp,NaN,DCEMS-RND 2,DCEMS-RND 2,OSUCOM,NaN
encounter_layout,NaN,NaN,First 11,NaN,First 11,First 11,First 11,NaN


In [9]:

for cn in file_stats_df.columns: print(cn, file_stats_df[cn].unique().shape[0])

file_name 426
is_a_one_triage_file 2
logger_version 2
is_scene_aborted 1
session_uuid 426
session_file_date 255
responder_name 235
responder_type 14
site_name 9
encounter_layout 3
